In [2]:
from pdata import PersonalizedMMUDataset, PersonalizedT2IDataset, get_personalized_mmu_dataloader, get_personalized_t2i_dataloader
from lightning.pytorch.utilities import CombinedLoader

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch.nn as nn
import numpy as np
from tqdm import tqdm
from PIL import Image

from models import Showo, MAGVITv2, get_mask_chedule
from training.prompting_utils import UniversalPrompting, create_attention_mask_predict_next, create_attention_mask_for_mmu
from training.utils import get_config, flatten_omega_conf, mask_or_random_replace_tokens, AverageMeter
from transformers import AutoTokenizer
from models.clip_encoder import CLIPVisionTower
from transformers import CLIPImageProcessor
from llava.llava import conversation as conversation_lib

conversation_lib.default_conversation = conversation_lib.conv_templates["phi1.5"]

import os
from omegaconf import DictConfig, ListConfig, OmegaConf
config = OmegaConf.load('configs/showo_demo.yaml')
# device setup
device = torch.device("cuda:6")

[2025-02-08 10:43:47,660] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.


/home/arc/miniconda3/envs/showo/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.2
 [WARNING]  using untested triton version (2.2.0), only 1.0.0 is known to be compatible


In [3]:
tokenizer = AutoTokenizer.from_pretrained(config.model.showo.llm_model_path, padding_side ="left")
uni_prompting = UniversalPrompting(tokenizer, max_text_len=config.dataset.preprocessing.max_seq_length,
                                       special_tokens=("<|soi|>", "<|eoi|>", "<|sov|>", "<|eov|>", "<|t2i|>", "<|mmu|>", "<|t2v|>", "<|v2v|>", "<|lvg|>"),
                                       ignore_id=-100, cond_dropout_prob=config.training.cond_dropout_prob)
vq_model = MAGVITv2.from_pretrained(config.model.vq_model.vq_model_name).to(device)
model = Showo.from_pretrained(config.model.showo.pretrained_model_path).to(device)

Working with z of shape (1, 13, 16, 16) = 3328 dimensions.
Look-up free quantizer with codebook size: 8192


The config attributes {'mask_token_id': 58497} were passed to Showo, but are not expected and will be ignored. Please verify your config.json configuration file.
/home/arc/Show-o/models/modeling_showo.py:49: FutureWarning: Accessing config attribute `w_clip_vit` directly via 'Showo' object attribute is deprecated. Please access 'w_clip_vit' over 'Showo's config object instead, e.g. 'unet.config.w_clip_vit'.
  if self.w_clip_vit:


attention implementation:  sdpa


In [4]:
# print(model.showo.get_input_embeddings())
model.showo.get_input_embeddings().num_embeddings
model.showo.get_input_embeddings().num_embeddings - len(tokenizer)
model.showo.get_input_embeddings().weight.data.shape
model.showo.lm_head.weight.shape
model.showo.lm_head.bias.shape

torch.Size([58498])

In [5]:
data_root = "/home/arc/full_mcdata"
concept = "dunpai"
nums_new_token_i = 16

In [6]:
#################################
new_tokens = [f"<{concept}>"] + [f"<token_{i}>" for i in range(nums_new_token_i)]
num_new_tokens = len(new_tokens)  # 17

# 已知的原始参数
# 文本 token 数量（ID 0-50304）
original_text_vocab_size = len(tokenizer)  
# Image token 数量（原 ID 50305-58497）
original_image_vocab_size = model.showo.get_input_embeddings().num_embeddings - len(tokenizer)

original_total_vocab = original_text_vocab_size + original_image_vocab_size  # 58498

# 新的参数
new_text_vocab_size = original_text_vocab_size + num_new_tokens  # 50305 + 17 = 50322
new_total_vocab = original_total_vocab + num_new_tokens          # 58498 + 17 = 58515

# ------------------------------
# Step 1: 修改 Tokenizer 的词汇表
# ------------------------------

# 添加新 token 到 50305-50321 的位置
num_new_tokens = tokenizer.add_tokens(new_tokens)
new_token_ids = tokenizer.convert_tokens_to_ids(new_tokens)
print("新 token ID:", new_token_ids)  # 应输出 50305-50321

# ------------------------------
# Step 2: 调整模型的权重
# ------------------------------
with torch.no_grad():
    # 获取嵌入层权重
    embeddings = model.showo.get_input_embeddings().weight.data
    
    # 扩展嵌入层（58498 -> 58515）
    model.showo.resize_token_embeddings(new_total_vocab)
    # new_embeddings = model.showo.get_input_embeddings().weight.data

    # 将原 Image Token 权重后移 17 位
    original_image_weights = embeddings[original_text_vocab_size:original_total_vocab].clone()
    model.showo.get_input_embeddings().weight.data[new_text_vocab_size:new_total_vocab] = original_image_weights
    
    # 初始化新 token 的权重（用原文本最后 17 个 token）
    # new_text_weights = embeddings[original_text_vocab_size - num_new_tokens : original_text_vocab_size].clone()
    # model.showo.get_input_embeddings().weight.data[original_text_vocab_size : new_text_vocab_size] = new_text_weights
    # print(model.showo.lm_head.weight.data.shape[1])
    # 处理 lm_head（假设与嵌入层共享权重）
    if model.showo.lm_head.weight.data.shape[0] == new_total_vocab:
        # 扩展 lm_head 权重
        lm_head = model.showo.lm_head
        new_lm_head = torch.nn.Linear(
            lm_head.in_features, 
            new_total_vocab, 
            bias=hasattr(lm_head, 'bias')
        )
        new_lm_head.weight.data = lm_head.weight.data.clone()
        new_lm_head.weight.data[new_text_vocab_size:new_total_vocab] = lm_head.weight.data[original_text_vocab_size:original_total_vocab]
        # new_lm_head.weight.data[original_text_vocab_size:new_text_vocab_size] = lm_head.weight.data[original_text_vocab_size - num_new_tokens : original_text_vocab_size]
        if hasattr(lm_head, 'bias'):
            new_lm_head.bias.data = lm_head.bias.data.clone()
            new_lm_head.bias.data[new_text_vocab_size:new_total_vocab] = lm_head.bias.data[original_text_vocab_size:original_total_vocab]
            # new_lm_head.bias.data[original_text_vocab_size:new_text_vocab_size] = lm_head.bias.data[original_text_vocab_size - num_new_tokens : original_text_vocab_size]
        
        model.showo.lm_head = new_lm_head
    else:
        raise ValueError("lm_head weights do not match the input embeddings!")

index_no_updates = torch.ones((new_total_vocab,), dtype=torch.bool)
index_no_updates[new_token_ids] = False
# ------------------------------
# 验证
# ------------------------------
# 检查新 token 的 ID
print("新增文本 token ID:", [tokenizer.convert_tokens_to_ids(t) for t in new_tokens])  # 应输出 50305-50321

# 检查一个原 Image Token 的新 ID
sample_image_token = tokenizer.convert_ids_to_tokens(original_text_vocab_size)  # 原 ID 50305
print(f"Concept Token '{sample_image_token}' 的新 ID:", tokenizer.convert_tokens_to_ids(sample_image_token))  # 应输出 50322

# 检查嵌入层形状
print("嵌入层大小:", model.showo.get_input_embeddings().weight.shape)  # 应显示 torch.Size([58515, 2048])

# 检查 index_no_updates 中 True 的位置和数量，True 应该是 new token ids
print("index_no_updates 中 False 的位置:", torch.nonzero(~index_no_updates).squeeze())  # 应输出 50305-50321
print("index_no_updates 中 True 的数量:", torch.sum(index_no_updates))  # 应输出 58498

with torch.no_grad():
    orig_embeds = model.showo.get_input_embeddings().weight.data.clone()
    orig_lm_head_weight = model.showo.lm_head.weight.data.clone()
    orig_lm_head_bias = model.showo.lm_head.bias.data.clone()

新 token ID: [50305, 50306, 50307, 50308, 50309, 50310, 50311, 50312, 50313, 50314, 50315, 50316, 50317, 50318, 50319, 50320, 50321]
新增文本 token ID: [50305, 50306, 50307, 50308, 50309, 50310, 50311, 50312, 50313, 50314, 50315, 50316, 50317, 50318, 50319, 50320, 50321]
Concept Token '<dunpai>' 的新 ID: 50305
嵌入层大小: torch.Size([58515, 2048])
index_no_updates 中 False 的位置: tensor([50305, 50306, 50307, 50308, 50309, 50310, 50311, 50312, 50313, 50314,
        50315, 50316, 50317, 50318, 50319, 50320, 50321])
index_no_updates 中 True 的数量: tensor(58498)


In [7]:
concept_embeds = model.showo.get_input_embeddings().weight.data[new_token_ids]
concept_lm_nead_wight = model.showo.lm_head.weight.data[new_token_ids]
concept_lm_nead_bias = model.showo.lm_head.bias.data[new_token_ids]

In [8]:
concept_embeds.shape, concept_lm_nead_wight.shape, concept_lm_nead_bias.shape

(torch.Size([17, 2048]), torch.Size([17, 2048]), torch.Size([17]))

In [9]:
uni_prompting.sptids_dict

{'<|soi|>': tensor([50296]),
 '<|eoi|>': tensor([50297]),
 '<|sov|>': tensor([50298]),
 '<|eov|>': tensor([50299]),
 '<|t2i|>': tensor([50300]),
 '<|mmu|>': tensor([50301]),
 '<|t2v|>': tensor([50302]),
 '<|v2v|>': tensor([50303]),
 '<|lvg|>': tensor([50304]),
 '<|sot|>': tensor([50256]),
 '<|eot|>': tensor([50256]),
 '<|pad|>': tensor([50295])}

In [10]:
vq_model.requires_grad_ = False
vq_model.eval()
model.train()
for names, p in model.named_parameters():
    if "embed_tokens" not in names and "lm_head" not in names:
        p.requires_grad = False
    else:
        p.requires_grad = True

trainable_params = [model.showo.get_input_embeddings().weight, model.showo.lm_head.weight, model.showo.lm_head.bias]
optimizer = torch.optim.AdamW(
            trainable_params, # for optimize the embeddings and the head
            lr=1e-2,
            betas=(0.9, 0.999),
            weight_decay=1e-2,
            eps=1e-08,
        )
for names, p in model.named_parameters():
    if p.requires_grad:
        print(f"{names} requires_grad") # embed_token, lm_head会更新

showo.model.embed_tokens.weight requires_grad
showo.lm_head.weight requires_grad
showo.lm_head.bias requires_grad


In [11]:
model.config.mask_token_id = model.showo.get_input_embeddings().num_embeddings - 1
model.mask_token_id = model.showo.get_input_embeddings().num_embeddings - 1

In [12]:
mask_schedule = get_mask_chedule(config.training.get("mask_schedule", "cosine"))
mask_id = model.mask_token_id
mask_dtype = model.showo.model.embed_tokens.weight.dtype

In [16]:

# t2i_dataset = PersonalizedT2IDataset(data_root, concept)
# t2i_dataloader = DataLoader(t2i_dataset, batch_size=5, shuffle=True, num_workers=10, pin_memory=True)

# mmu_dataloader = get_personalized_mmu_dataloader(data_root, concept, tokenizer, batch_size=5, num_workers=0, max_length=128)
t2i_dataloader = get_personalized_t2i_dataloader(data_root, concept, tokenizer, batch_size=2, num_workers=0, max_length=128)


# iterables = {
#     # 'mmu_flow': mmu_dataloader,
#     't2i_flow': t2i_dataloader
# }


# combined_dataloader = CombinedLoader(iterables, mode="max_size_cycle")

# Before adding the new tokens, the vocab size is 58498
# vocab size = 58498 = 50295  llm vocabsize
#                    + 10     <|soi|> <|eoi|> <|sov|> <|eov|> <|t2i|> <|mmu|> <|t2v|> <|v2v|> <|lvg|> <|pad|>
#                    + 8192   vq model codebook size
#                    + 1      mask token (token id == 58497)
from typing import Union


uni_prompting.sptids_dict
# {'<|soi|>': tensor([50296]),
#  '<|eoi|>': tensor([50297]),
#  '<|sov|>': tensor([50298]),
#  '<|eov|>': tensor([50299]),
#  '<|t2i|>': tensor([50300]),
#  '<|mmu|>': tensor([50301]),
#  '<|t2v|>': tensor([50302]),
#  '<|v2v|>': tensor([50303]),
#  '<|lvg|>': tensor([50304]),
#  '<|sot|>': tensor([50256]),
#  '<|eot|>': tensor([50256]),
#  '<|pad|>': tensor([50295])}

# uni_prompting.text_tokenizer == tokenizer
def prepare_inputs_and_labels(
        pixel_values_or_image_ids: Union[torch.FloatTensor, torch.LongTensor],
        texts: Union[str, str],
        min_masking_rate: float = 0.0,
        is_train: bool = True,
):

    image_tokens = vq_model.get_code(pixel_values_or_image_ids)
    image_tokens = image_tokens + len(uni_prompting.text_tokenizer)

    # create MLM mask and labels
    input_ids, labels, loss_weight, mask_prob = mask_or_random_replace_tokens(
        image_tokens,
        mask_id,
        config,
        mask_schedule=mask_schedule,
        is_train=is_train,
    )
    input_ids, masks, labels = uni_prompting((texts, input_ids, labels), 't2i')

    return input_ids, labels, mask_prob, image_tokens

In [17]:
model.output_size = new_total_vocab
save_path = os.path.join("saves", concept, "only_t2i_v4")
os.makedirs(save_path, exist_ok=True)
global_loss_list = []
for epoch in range(0, 1000):
    print(f"Epoch {epoch+1}")
    loss_list = []
    loss_t2i_list = []
    # loss_mmu_list = []
    for batch in tqdm(t2i_dataloader):
        # batch_size_mmu = batch["mmu_flow"]["images"].shape[0]
        batch_size_t2i = batch["images"].shape[0]
        
        # t2i format
        pixel_values, texts = batch["images"], batch["conditions"]
        pixel_values = pixel_values.to(device)
        input_ids, labels, mask_prob, image_tokens_ori = prepare_inputs_and_labels(pixel_values, texts, is_train=True)
        attention_mask = create_attention_mask_predict_next(input_ids,
                                                                pad_id=int(uni_prompting.sptids_dict['<|pad|>']),
                                                                soi_id=int(uni_prompting.sptids_dict['<|soi|>']),
                                                                eoi_id=int(uni_prompting.sptids_dict['<|eoi|>']),
                                                                rm_pad_in_image=True,
                                                                return_inverse_mask=True)
        attention_mask = attention_mask.to(mask_dtype)
        # 美国队长的盾牌
        
        # mmu format
        # pixel_values_mmu, input_ids_mmu, labels_mmu = (batch["mmu_flow"]["images"],
        #                                               batch["mmu_flow"]["input_ids"],
        #                                               batch["mmu_flow"]["labels"])
        # pixel_values_mmu = pixel_values_mmu.to(device, non_blocking=True)
        # input_ids_mmu = input_ids_mmu.to(device, non_blocking=True)
        # image_tokens_mmu = vq_model.get_code(pixel_values_mmu)
        # image_tokens_mmu = image_tokens_mmu + len(uni_prompting.text_tokenizer)
        
        # input_ids_mmu = torch.cat([
        #             (torch.ones(input_ids_mmu.shape[0], 1) * uni_prompting.sptids_dict['<|mmu|>']).to(
        #                 device),
        #             (torch.ones(input_ids_mmu.shape[0], 1) * uni_prompting.sptids_dict['<|soi|>']).to(
        #                 device),
        #             image_tokens_mmu,
        #             (torch.ones(input_ids_mmu.shape[0], 1) * uni_prompting.sptids_dict['<|eoi|>']).to(
        #                 device),
        #             input_ids_mmu,
        #         ], dim=1).long()

        # labels_mmu = torch.cat([
        #             (torch.ones(input_ids_mmu.shape[0], 1) * uni_prompting.ignore_id).to(device),
        #             (torch.ones(input_ids_mmu.shape[0], 1) * uni_prompting.ignore_id).to(device),
        #             torch.ones_like(image_tokens_mmu) * uni_prompting.ignore_id,
        #             (torch.ones(input_ids_mmu.shape[0], 1) * uni_prompting.ignore_id).to(device),
        #             labels_mmu.to(device)
        #         ], dim=1).long()
        
        
        # attention_mask_mmu = create_attention_mask_for_mmu(input_ids_mmu.to(input_ids.device),
        #                                                        eoi_id=int(uni_prompting.sptids_dict['<|eoi|>']))
        # attention_mask_mmu = attention_mask_mmu.to(mask_dtype)
        # attention_mask = torch.cat([attention_mask, attention_mask_mmu], dim=0)
        # input_ids = torch.cat((input_ids, input_ids_mmu.to(input_ids.device)), dim=0)
        # labels = torch.cat((labels, labels_mmu.to(input_ids.device)), dim=0)
        
        optimizer.zero_grad()
        
        logits, loss_t2i, loss_lm, loss_mmu = model(
                    input_ids=input_ids,
                    input_embeddings=None,
                    attention_mask=attention_mask,
                    labels=labels,
                    label_smoothing=0.0,
                    batch_size_t2i=batch_size_t2i,
                    batch_size_lm=0,
                    batch_size_mmu=0,
                    max_seq_length=128,
                )
        # loss = 0.8 * loss_t2i + 0.2 * loss_mmu
        loss = loss_t2i
        loss.backward()
        optimizer.step()
        loss_list.append(loss.item())
        loss_t2i_list.append(loss_t2i.item())
        # loss_mmu_list.append(loss_mmu.item())
        # tqdm.set_postfix(loss=loss.item(), loss_t2i=loss_t2i.item(), loss_mmu=loss_mmu.item())
        # tqdm.write(f"loss: {loss.item()}, loss_t2i: {loss_t2i.item()}, loss_mmu: {loss_mmu.item()}")
        # 恢复原始权重
        with torch.no_grad():
            model.showo.get_input_embeddings().weight.data[index_no_updates] = orig_embeds[index_no_updates]
            model.showo.lm_head.weight.data[index_no_updates] = orig_lm_head_weight[index_no_updates]
            model.showo.lm_head.bias.data[index_no_updates] = orig_lm_head_bias[index_no_updates]
    print(f"Epoch {epoch+1} loss: {np.mean(loss_list)}, loss_t2i: {np.mean(loss_t2i_list)}")
    # print(f"  Token-Norm: {model.showo.get_input_embeddings().weight[new_token_ids].norm().item()}")
    # print(f"  index_no_updates-Token-Norm: {model.showo.get_input_embeddings().weight[index_no_updates].norm().item()}")
    # print(f"  LM-Head-Weight-Norm: {model.showo.lm_head.weight[new_token_ids].norm().item()}")
    # print(f"  index_no_updates-LM-Head-Weight-Norm: {model.showo.lm_head.weight[index_no_updates].norm().item()}")
    # print(f"  LM-Head-Bias-Norm: {model.showo.lm_head.bias[new_token_ids].norm().item()}")
    # print(f"  index_no_updates-LM-Head-Bias-Norm: {model.showo.lm_head.bias[index_no_updates].norm().item()}")
    global_loss_list.append(np.mean(loss_list))
    
    # print(f"Epoch {epoch} loss: {np.mean(loss_list)}, loss_t2i: {np.mean(loss_t2i_list)}")
    if (epoch+1) % 100 == 0:
        save_path_embed = os.path.join(save_path, f"epoch_{epoch+1}_embed.pt")
        save_path_lm_head_weight = os.path.join(save_path, f"epoch_{epoch+1}_lm_head_weight.pt")
        save_path_lm_head_bias = os.path.join(save_path, f"epoch_{epoch+1}_lm_head_bias.pt")
        
        torch.save(model.showo.get_input_embeddings().weight.data[new_token_ids], save_path_embed)
        torch.save(model.showo.lm_head.weight.data[new_token_ids], save_path_lm_head_weight)
        torch.save(model.showo.lm_head.bias.data[new_token_ids], save_path_lm_head_bias)
        
        

Epoch 1


100%|██████████| 5/5 [00:01<00:00,  2.67it/s]


Epoch 1 loss: 5.883703804016113, loss_t2i: 5.883703804016113
Epoch 2


100%|██████████| 5/5 [00:01<00:00,  2.61it/s]


Epoch 2 loss: 6.936676502227783, loss_t2i: 6.936676502227783
Epoch 3


...

Epoch 998 loss: 5.367235469818115, loss_t2i: 5.367235469818115
Epoch 999


100%|██████████| 5/5 [00:01<00:00,  2.87it/s]


Epoch 999 loss: 6.181424236297607, loss_t2i: 6.181424236297607
Epoch 1000


100%|██████████| 5/5 [00:01<00:00,  2.74it/s]

Epoch 1000 loss: 5.0533905029296875, loss_t2i: 5.0533905029296875


In [26]:
global_loss_list

array([5.01643962, 5.09258044, 4.93272986, 4.958125  , 4.97583602,
       5.24311753, 4.98709406, 4.95030022, 5.38294747, 4.98889398,
       5.49991792, 4.82176958, 5.17182683, 5.21638189, 5.05373831,
       5.29155654, 4.89407912, 5.06958536, 5.1037549 , 4.69237597,
       5.03856981, 4.98109551, 4.6533004 , 5.02418197, 4.94685748,
       5.36309365, 4.92077034, 5.12259176, 4.721271  , 4.65606331,
       4.60750671, 4.75911181, 5.21908968, 4.87084234, 5.00624558,
       4.83370509, 5.08076675, 4.82779905, 4.84456172, 4.60004929,
       5.08952033, 4.42633574, 5.0397859 , 4.43144518, 5.18609002,
       4.95075648, 5.04640392, 4.83458514, 5.17871941, 5.05027549,
       5.12566279, 4.54941443, 4.81944222, 4.91083594, 4.82251689,
       4.63891494, 4.76838633, 4.80793931, 4.93792125, 4.47628796,
       4.69185613, 4.93935534, 4.78203216, 4.7871423 , 5.06824703,
       4.58363946, 4.32588878, 4.70750208, 4.97092187, 4.77463023,
       4.83224711, 4.71225255, 4.64119231, 4.55381514, 4.71156

In [18]:
# 将长度为 960 的 global_loss_list 每10个元素求平均，得到 96 个元素的列表
global_loss_list

[5.883703804016113,
 6.936676502227783,
 6.207831144332886,
 6.371516132354737,
 7.15727014541626,
 6.654879760742188,
 6.030917835235596,
 5.889976358413696,
 5.770388793945313,
 7.479159355163574,
 6.65997838973999,
 7.088486289978027,
 5.650168609619141,
 5.584375095367432,
 5.71863317489624,
 6.119616413116455,
 6.030371952056885,
 5.830107307434082,
 6.530434799194336,
 7.14695110321045,
 5.385854911804199,
 7.129662132263183,
 6.35681562423706,
 6.238932418823242,
 6.035948467254639,
 6.602953910827637,
 6.979196071624756,
 6.630860710144043,
 6.756623363494873,
 5.188763570785523,
 6.695420169830323,
 6.050587224960327,
 6.393011617660522,
 6.030512571334839,
 6.051673221588135,
 6.413208675384522,
 6.413530111312866,
 5.8468482971191404,
 6.3862982273101805,
 5.953164005279541,
 5.562509822845459,
 6.406403303146362,
 5.938363742828369,
 6.735985469818115,
 6.354169845581055,
 6.828611850738525,
 6.883069801330566,
 5.677963829040527,
 5.671456956863404,
 6.627357006072998,
 5.

In [20]:
global_loss_list_np = np.array(global_loss_list)
global_loss_list_np = global_loss_list_np.reshape(-1, 10)
global_loss_list_np = global_loss_list_np.mean(axis=1)
g = global_loss_list_np.tolist()

In [22]:
g

[6.438231983184815,
 6.235912313461304,
 6.330561118125916,
 6.223425412178039,
 6.268589162826538,
 6.299460434913636,
 6.145899701118469,
 6.156520028114318,
 5.988226685523987,
 6.007852244377136,
 6.260523662567138,
 6.387895357608795,
 6.4752106809616095,
 6.33093533039093,
 5.8345703649520875,
 6.296097173690795,
 6.272945008277892,
 5.949320709705352,
 5.9288853788375855,
 6.066922388076782,
 5.776312074661254,
 6.077973670959472,
 6.087599296569824,
 6.2798117780685425,
 6.346329197883606,
 6.383945407867431,
 6.262394089698791,
 6.256709742546081,
 6.267494511604309,
 6.314464588165283,
 6.191693062782288,
 5.970439968109131,
 6.202673540115357,
 5.971730904579163,
 6.217498068809509,
 5.984234366416931,
 5.888112373352051,
 6.342722511291504,
 6.075039405822754,
 5.913822050094605,
 6.084694442749023,
 6.365543718338013,
 5.949003896713258,
 6.008516116142274,
 5.846461505889893,
 6.025424890518188,
 6.265800604820251,
 5.91506160736084,
 6.0341055560112,
 5.8942959070205685,